In [1]:
import pandas as pd
import pickle
import numpy as np
import re
from datetime import datetime
from dateutil import parser
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn import linear_model
import sklearn.metrics as sklm
import statsmodels.api as sm
import numpy.random as nr
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df.reset_index(inplace = True, drop = True)

df['date'] = pd.DatetimeIndex(pd.to_datetime(df.date))

df.set_index(['teams', 'date'], inplace = True)
df.sort_index(inplace = True)

df['home_goals'] = np.nan
df['away_goals'] = np.nan

df.loc[df.p_home == 1, 'home_goals'] = df.loc[df.p_home == 1, 'team_goals']
df.loc[df.p_home == 0, 'away_goals'] = df.loc[df.p_home == 0, 'team_goals']

df['away_goals'] = df.groupby(['league', 'season', 'match_number'])['away_goals'].transform('max')
df['home_goals'] = df.groupby(['league', 'season', 'match_number'])['home_goals'].transform('max')

df.loc[df.p_home == 1, 'opp_goals'] = df.loc[df.p_home == 1, 'away_goals']
df.loc[df.p_home == 0, 'opp_goals'] = df.loc[df.p_home == 0, 'home_goals']

df['win'] = 0
df['loss'] = 0

df.loc[df.results == 'W', 'win'] = 1
df.loc[df.results == 'L', 'loss'] = 1

In [ ]:
pickle.dump(df, open("player_level.pckl", "wb" ) )

# Merge with old data

In [2]:
old_df = pickle.load(open('player_level_old.pckl', 'rb'))

C:\Users\Schmidt\Anaconda3\lib\site-packages\pandas\core\indexes\base.py:162: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return cls.__new__(cls, **d)


In [5]:
old_df.league.unique()

array(['Serie_A', 'Premier League', 'Bundesliga'], dtype=object)

In [6]:
old_df.loc[old_df.league == 'Bundesliga', 'season'].unique()

array(['11_12', '12_13', '13_14', '14_15', '15_16', '16_17', '09_10',
       '10_11'], dtype=object)

In [ ]:
columns_to_drop = ['p_sub_out',
 'p_penalty_missed_times',
 'team_goals',
 'p_away',
 'win',
 'p_assist_times',
 'p_home',
 'p_started_game',
 'p_error_times',
 'p_event',
 'home_goals',
 'p_team',
 'p_yellow_card_times',
 'match_number',
 'p_own_goal_times',
 'p_sub_in',
 'loss',
 'opp_goals',
 'p_tackle_last_man_times',
 'p_index',
 'p_event_times',
 'p_red_card_time',
 'p_opponent',
 'p_goal_times',
 'p_penalty_saved_times',
 'away_goals',
 'p_post_crossbar_times']

old_df.drop(columns_to_drop, axis = 1, inplace = True)

rename_dict = {'p_clearances': 'p_clears', 'p_long_ball': 'p_long_balls', 'p_dispossessed': 'p_disp', 
               'p_clear_off_line': 'p_clears_off_line', 'p_acc': 'p_pass_acc', 'p_blocked_shots': 'p_blocks',
               'p_acc_through_ball': 'p_acc_through_balls', 'p_minutes_played': 'p_min_played', 'p_error': 'p_errors',
               'p_tackle_last_man': 'p_tackles_last_man', 'p_through_ball': 'p_through_balls',
               'p_bad_control': 'p_turnovers', 'p_acc_long_ball': 'p_acc_long_balls', 'p_bad_control': 'p_bad_controls',
               'p_results': 'p_result', }

old_df.rename(index = str, columns = rename_dict, inplace = True)

list(set(old_df.columns) - set(df.columns))

list(set(df.columns) - set(old_df.columns))